# Restmateriaal

We onderzoeken hier verschillende mogelijkheden voor het maken van een dergelijke gateway:

* computer (bijv. Raspberry Pi) via USB verbonden met een microbit
* microbit met WiFi radio (elecfreaks iot-bit

De Elecfreaks iot-bit wordt gewoonlijk gebruikt om een enkele microbit te verbinden met het internet. Maar we kunnen deze ook programmeren als gateway, om een lokaal microbit-netwerk te verbinden met het internet.

Voor het gebruik van een gateway in een *schoolnetwerk* moet deze aan een aantal extra eisen voldoen:

* geen gebruik van andere poorten dan 80 en 433 (http, https); veel andere poorten en protocollen zijn vaak geblokkeerd;
* voor een WiFi verbinding: de mogelijkheid om te verbinden met een netwerk met een wachtwoord per computer (of per gebruiker), in plaats van een wachtwoord voor het hele netwerk.
* (een alternatief is het gebruik van een bedraad netwerk; dan heb je niet het probleem dat je moet inloggen op het netwerk.

Voor het verbinden van de IoT-gateway met het internet gebruiken we meestal het MQTT protocol; dit biedt symmetrische "publish-subscribe" communicatie mogelijk, in tegenstelling tot de client-server communicatie van HTTP.
Het MQTT protocol gebruikt gewoonlijk een TCP-verbinding via de poorten 1883/1884. Maar het is tegenwoordig ook mogelijk om een "websockets" (HTTP) verbinding te gebruiken als transportlaag voor MQTT. Daarmee omzeil je de poort-beperkingen van een schoolnetwerk.

---

## Uit het overzicht.

### Bijeenkomst 1

* wat is het internet of things? - als ontwikkeling van het internet, na het web.
* waarom het internet of things? - slimme oplossingen, betere beslissingen
* hoe het internet of things? - onderdelen en hun verbindingen, relaties
    * eerst als functionele onderdelen ("eindapparaten" vanuit het netwerk gezien)
    * dan met verbindingen en verbindingsapparaten
* opbouw van een IoT-knoop - 
* protocollen: MQTT;
* MQTT - publish/subscribe principe; topic, payload keuzes (JSON)
* draadloze IoT-knopen: keuze van een radio
    * hier: microbit-radio (met eigenschappen)

Praktische opdrachten:

* MQTT - publish/subscribe (oefenen met MQT3)
* NodeRed - 
* microbit IoT-knoop, microbit (pakket) radio
    * programmeren van de microbit IoT-knoop (te volgen via host)
    * programmeren van de logger-knoop (als beschikbaar
    * LPP formaat; en (via logger) JSON formaat
    * volgende keer: verbinden aan het internet (MQTT)

### Bijeenkomst 2

* gateway: functie, opbouw; protocollen
* MQTT: adressering van knopen, (sensoren?); netwerk?; 

Praktische opdrachten:

* programmeren (installeren) van de gateway; experimenteren met het lokale netwerk en MQTT
* NodeRed: monitoring van het IoT-netwerk
* NodeRed: MQTT verbindingen; publish, subscribe in NodeRed
* NodeRed: interpreteren van JSON-berichten; dashboard

### Bijeenkomst 3

* (automatisch) sturen: controllers
    * NodeRed als voorbeeld
* koppelen van sensoren en actuatoren; sturen

Praktische opdrachten:

* NodeRed: (automatisch) sturen (condities in flows)

### Open punten

* user interface, dashboard
* opslag van sensor-gegevens
    * per knoop? of per sensor? (of allebeide?)
* identificatie van sensoren - (en van knopen); op het niveau van de toepassing
* 

### Opdrachten

Eerste bijeenkomst:

* kennismaking MQTT (vooral via web-app)
* kennismaking JSON (idem; )
* kennismaking NodeRed (met JSON, MQTT berichten)
* kennismaking LPP (binaire codering)

| opdracht | omschrijving |
|:--       | :---         |
| **mqtt**     |  |
| x.x      | chat-0: mqtt publish/subscribe via web-app |
| x.x      | chat-1: mqtt publish/subscribe, met topic en wildcard |
| x.x.     | sensors en actuators: mqtt publish/subscribe, via web-app |
| **nodered** | |
| b.b      | ontvangen en verwerken van elementaire berichten, koppelen |
| b.b      | ontvangen van mqtt-berichten (nb: in JSON) |
| **lpp**         |              |
| a.a      | lpp gecodeerde berichten via pakketradio (radio logger) |
| a.a      | lpp omgezet naar json (radio logger) |

NodeRed is een *controller* waarmee je de verschillende onderdelen (en protocollen) van de IoT-keten aan elkaar kunt koppelen.

Tweede bijeenkomst:

* nadruk op de gateway,
* verwerken van de sensordata (dashboard?)
* omzetten van JSON in LPP (v.v.)

| opdracht | omschrijving |
|:--       | :---         |
| **mqtt**     |  |
| x.x      | chat-0: mqtt publish/subscribe via web-app |
| x.x      | chat-1: mqtt publish/subscribe, met topic en wildcard |
| x.x.     | sensors en actuators: mqtt publish/subscribe, via web-app |
| **nodered** | |
| b.b      | ontvangen en verwerken van elementaire berichten, koppelen |
| b.b      | ontvangen van mqtt-berichten (nb: in JSON) |
| **lpp**         |              |
| a.a      | lpp gecodeerde berichten via pakketradio (radio logger) |
| a.a      | lpp omgezet naar json (radio logger) |

Derde bijeenkomst:

* sturen met NodeRed
* 

| opdracht | omschrijving |
|:--       | :---         |
| **mqtt**     |  |
| x.x      | chat-0: mqtt publish/subscribe via web-app |
| x.x      | chat-1: mqtt publish/subscribe, met topic en wildcard |
| x.x.     | sensors en actuators: mqtt publish/subscribe, via web-app |
| **nodered** | |
| b.b      | ontvangen en verwerken van elementaire berichten, koppelen |
| b.b      | ontvangen van mqtt-berichten (nb: in JSON) |
| **lpp**         |              |
| a.a      | lpp gecodeerde berichten via pakketradio (radio logger) |
| a.a      | lpp omgezet naar json (radio logger) |

---

## microbit IoT-node

Onderstaande code maakt gebruik van de ulpp-library: die gebruikt teveel geheugen voor de microbit V1.

In [1]:
from microbit import *
from ulpp import LppFrame, bytes_to_dict
import radio
import machine
from utime import *

def get_serial_number() -> int:
    NRF_FICR_BASE = 0x10000000
    DEVICEID_INDEX = 25 # deviceid[1]
    return machine.mem32[NRF_FICR_BASE + (DEVICEID_INDEX*4)] & 0xFFFFFFFF

nodeID = get_serial_number() & 0xFFFF
counter = 0

uplink_tag = 0x0A
downlink_tag = 0x0B

led0 = 0
led1 = 0 # not used

def handle_actuators(data):
    lpp = bytes_to_dict(data)
    if 0 in lpp and 'dOut' in lpp[0]:
        led0 = lpp[0]['dOut']
        pin0.write_digital(led0)
        if led0 == 1:
            display.show(Image.HAPPY)
        else:
            display.show(Image.SAD)
    if 1 in lpp and 'dOut' in lpp[1]:
        led1 = lpp[1]['dOut']
        pin1.write_digital(led0)

def send_sensors(buttonA, buttonB):
    global counter
    
    print("send")
    (chi, clo) = divmod(counter, 256)
    (nhi, nlo) = divmod(nodeID, 256)
    prefix = bytes([uplink_tag, nhi, nlo, chi, clo])
    counter += 1
  
    frame = LppFrame(data=prefix)
    frame.add_digital_output(0, led0)
    frame.add_digital_output(1, led1)
    frame.add_digital_input(2, buttonA)
    frame.add_digital_input(3, buttonB)
    frame.add_temperature(4, temperature()*10)
    frame.add_analog_input(8, display.read_light_level())
    buffer = frame.to_bytes()
    radio.send_bytes(buffer) 
    
timer_period = 60000
timer_deadline = ticks_add(ticks_ms(), timer_period)

display.scroll(hex(nodeID))
print(hex(nodeID))

radio.on()
send_sensors(button_a.is_pressed(), button_b.is_pressed())
while True:
    if button_a.was_pressed():
        send_sensors(1, button_b.is_pressed())
        
    if button_b.was_pressed():
        send_sensors(button_a.is_pressed(), 1)
        
    if ticks_diff(timer_deadline, ticks_ms()) <= 0:
        send_sensors(button_a.is_pressed(), button_b.is_pressed())
        timer_deadline = ticks_add(timer_deadline, timer_period)
        
    rec_bytes = radio.receive_bytes()
    if rec_bytes != None:
        if rec_bytes[0] == downlink_tag:
            rec_nodeID = rec_bytes[1] * 256 + rec_bytes[2]
            if nodeID == rec_nodeID:  # msg for this node
                display.show('R')
                rec_counter = rec_bytes[3] * 256 + rec_bytes[4]
                handle_actuators(rec_bytes[5:])
                send_sensors(button_a.is_pressed(), button_a.is_pressed())

    sleep_ms(10)

ModuleNotFoundError: No module named 'microbit'

## Identificatie van een microbit-knoop

Als identificatie voor een IoT-knoop gebruiken we een 16-bits getal, in JSON en in MQTT topics weergegeven als hexadecimale string.

> In het geval van WiFi-knopen gebruiken we daarvoor de laatste 16 bits van het WiFi MAC adres.

Een microbit heeft een unieke identificatie: we kunnen daarvan de laatste 16 bits gebruiken.

```
from microbit import *
import machine

def get_serial_number(type=hex):
    NRF_FICR_BASE = 0x10000000
    DEVICEID_INDEX = 25 # deviceid[1]
    return type(machine.mem32[NRF_FICR_BASE + (DEVICEID_INDEX*4)]& 0xFFFFFFFF)
```

In [7]:
hex(20000)[-4:]

'4e20'

NB: in de LPP-communicatie gebruiken we de identificatie als getal!

NB:

* er is een verschil tussen het decoderen van binaire LPP in een IoT-knoop, met als doel het aansturen van de actuatoren; en het decoderen van binaire LPP in de gateway, met als doel de omzetting in JSON. In dit laatste geval moet je alle elementen decoderen, in het eerste geval alleen maar de actuatoren.
* bij het decoderen in de IoT-knoop moet de inhoud overeenkomen met de actuele actuatoren van de IoT-knoop. Hoe controleer je dit?
    * het channel bepaalt al met welke actuator je te maken hebt. 
* worden de sensoren geskipt?
* in de IoT-knoop heb je (in principe) alleen te maken met gehele getallen als waarden voor de sensoren en actuatoren.


Decoderen van LPP in een IoT-knoop, omzetten in actuator-acties:

In [ ]:
in_buffer = bytearray([])
in_pos = 0

def get_byte() -> int:
    global in_pos
    byte = in_buffer[in_pos]
    in_pos += 1
    return byte

def get_int() -> int:
    hi = get_byte()
    lo = get_byte()
    return hi * 256 + lo

# NB: onderstaande functies zijn niet generiek,
# maar specifiek voor de IoT-knoop.

def handle_led(channel: int, value: int):
    if channel == 0:
        pin0.write_digital(value)
        if value == 0:
            display.show(off_image)
        else:
            display.show(on_image)
    elif channel == 1:
        pin1.write_digital(value)

def handle_lpp_actuators():  # naam?
    while in_pos < len(in_buffer):
        channel = get_byte()
        tag = get_byte()
        if (channel == 0 or channel == 1) and tag == LPP_dOut:
            handle_led(channel, get_byte())
        elif tag == LPP_dIn or tag == LPP_presence or tag == LPP_humidity:
            skip = get_byte()
        else:
            skip = get_int()

Coderen van LPP in IoT-knoop.

Ook dit hoeft niet helemaal generiek te zijn: je hoeft alleen voor de aanwezige sensoren (en actuatoren?) te coderen.

In [ ]:
out_buffer = bytearray([])
out_pos = 0

def put_byte(value: int):
    out_buffer[out_pos] = value
    out_pos += 1

def put_int(value: int):
    while value < 0:
        value = value + 65536
    put_byte(value // 256)
    put_byte(value % 256)

def put_sensor_byte(channel: int, tag: int, value: int):
    put_byte(channel)
    put_byte(tag)
    put_byte(value)
    
def put_sensor_int(channel: int, tag: int, value: int)
    put_byte(channel)
    put_byte(tag)
    put_int(value)

def send_sensors(button0: int, button1:int, presence: int):
    put_sensor_byte(0, LPP_dOut, pin0.read_digital()) # or: led0
    put_sensor_byte(1, LPP_dOut, pin1.read_digital()) # or: led1
    put_sensor_byte(2, LPP_dIn, button0)
    put_sensor_byte(3, LPP_dIn, button1)
    put_sensor_int(4, LPP_temperature, machine.temperature() * 10)
    put_sensor_byte(7, LPP_presence, presence)
    put_sensor_int(8, LPP_aOut, display.read_light_level())

## ulpp - vorige versies

In [ ]:
class LppFrame(object):

    # some assumptions:
    # - the value-parameters are in the LPP-required format and range
    #   i.e. scaling is done by the caller (if needed)
    #   all value-parameters are int
    
    def __init__(self, data=b'', maxsize=32):
        self.buffer = bytearray(data)
        self.maxsize = maxsize
        self.pos = 0

    def __str__(self):
        return str(list(self.buffer))
    
    def to_bytes(self):
        return self.buffer
    
    def add_byte(self, channel, tag, value):
        if len(self.buffer) + 3 > self.maxsize:
            raise OverflowError
        self.buffer.append(channel)
        self.buffer.append(tag)
        self.buffer.append(value)

    def add_unsigned_int16(self, channel, tag, value):
        if len(self.buffer) + 4 > self.maxsize:
            raise OverflowError        
        self.buffer.append(channel)
        self.buffer.append(tag)
        if value >= 65536:
            value = value % 65536
        (hi, lo) = divmod(value, 256)
        self.buffer.append(hi)
        self.buffer.append(lo)

    def add_signed_int16(self, channel, tag, value):
        while value < 0:
            value = value + 65536
        self.add_unsigned_int16(channel, tag, value)

    def add_digital_input(self, channel, value):
        self.add_byte(channel, 0, value)

    def add_digital_output(self, channel, value):
        self.add_byte(channel, 1, value)
        
    def add_analog_input(self, channel, value):
        self.add_signed_int16(channel, 2, value)

    def add_analog_output(self, channel, value):
        self.add_signed_int16(channel, 3, value)

    def add_luminosity(self, channel, value):
        self.add_unsigned_int16(channel, 101, value)

    def add_presence(self, channel, value):
        self.add_byte(channel, 102, value)

    def add_temperature(self, channel, value):
        # temperature: 0.1C, signed int
        self.add_signed_int16(channel, 103, value)

    def add_humidity(self, channel, value):
        # rel. humidity: 0.5% unsigned byte
        self.add_byte(channel, 104, value)

    def add_barometer(self, channel, value):
        # barometric pressue: 0.1 hPa unsigned int16
        self.add_unsigned_int16(channel, 115, value)

def bytes_to_dict(data):
    
    buffer = data
    pos = 0

    def nextbyte():
        nonlocal pos
        if pos >= len(buffer):
            raise OverflowError
        value = buffer[pos]
        pos = pos + 1
        return value
    
    def nextunsignedint():
        hi = nextbyte()
        lo = nextbyte()
        value = hi * 256 + lo
        return value 
    
    def nextint():
        value = nextunsignedint()
        if value > 32767:
            value = value - 65536
        return value
    

    pos = 0
    obj = {}

    while pos < len(buffer):
        channel = nextbyte()
        tag = nextbyte()
        if tag == 0:
            value = nextbyte()
            obj[channel] = {'dIn': value}
        elif tag == 1:
            value = nextbyte()
            obj[channel] = {'dOut': value}                
        elif tag == 2:
            value = nextint()
            obj[channel] = {'aIn': value}                
        elif tag == 3:
            value = nextint()
            obj[channel] = {'aOut': value}
        elif tag == 101:
            value = nextunsignedint()
            obj[channel] = {'luminosity': value}                
        elif tag == 102:
            value = nextbyte()
            obj[channel] = {'presence': value}                
        elif tag == 103:
            value = nextint()                
            obj[channel] = {'temperature': value}                
        elif tag == 104:
            value = nextbyte()
            obj[channel] = {'humidity': value}                
        elif tag == 115:
            value = nextunsignedint()
            obj[channel] = {'barometer': value}
            
    return obj

def lpp_to_json(data: bytearray) -> str:
    
    pos = 0
    
    def nextbyte():
        nonlocal pos
        if pos >= len(data):
            raise OverflowError
        value = data[pos]
        pos = pos + 1
        return value
    
    def nextunsignedint():
        hi = nextbyte()
        lo = nextbyte()
        return hi * 256 + lo 
    
    def nextint():
        value = nextunsignedint()
        if value > 32767:
            value = value - 65536
        return value
    
    pos = 0
    json_string = '{'
    sep = ''

    while pos < len(data):
        channel = nextbyte()
        tag = nextbyte()
        if tag == 0:
            value = nextbyte()
            tagname = 'dIn'
        elif tag == 1:
            value = nextbyte()
            tagname = 'dOut'
        elif tag == 2:
            value = nextint()
            tagname = 'aIn'
        elif tag == 3:
            value = nextint()
            tagname = 'aOut'
        elif tag == 101:
            value = nextunsignedint()
            tagname = 'luminosity'
        elif tag == 102:
            value = nextbyte()
            tagname = 'presence'
        elif tag == 103:
            value = nextint()                
            tagname = 'temperature'
        elif tag == 104:
            value = nextbyte()
            tagname = 'humidity'
        elif tag == 115:
            value = nextunsignedint()
            tagname = 'barometer'
        json_string += '{0}"{1}": {{"{2}": {3}}}'.format(sep, channel, tagname, value)
        sep = ', '
        
    json_string += '}' 
    return json_string

def dict_to_bytes (obj):
    lpp = LppFrame()

    for channel in obj:
        item = obj[channel]  # item is an object with a single key...
        if type(channel) is str:
            channel = int(channel)
        for key in item:
            if key == 'dOut':
                lpp.add_digital_output(channel,  item[key])
            elif key == 'aOut':
                lpp.add_analog_output(channel,  item[key])
            else:
                # not implemented, raise exception? only output allowed
                raise ValueError('only output values allowed in actuator msg')

    return lpp.to_bytes()

## Bytes en bytearrays

Het gebruik van `bytearray` is belangrijk als je efficiënt met het microbit-geheugen wilt omgaan, in het bijzonder op de microbit V1. (Zie: )

Een Python `bytes` waarde is immutable: je kunt deze als gehele waarde gebruiken, of indiceren. Het resultaat van indicering is een `int` (!).

De microbit radio-berichten worden ontvangen en verstuurd als `bytes` waarden.

In [30]:
b = b'abc'

In [31]:
type(b)

bytes

In [32]:
b[1]

98

In [33]:
type(b[1])

int

In [40]:
b + b

b'abcabc'

Een Python `bytearray` is een *mutable* waarde: je kunt elementen daarvan veranderen, als in een normaal array. De waarden moeten in het bereik 0..255 liggen, in Python termen: in `range(0, 256)`.

Je kunt een `bytearray` initialiseren met een `bytes` waarde; je kunt een `bytearray ba` ook omzetten in een `bytes`-waarde: `bytes(ba)`.



In [34]:
ba = bytearray(b)

In [35]:
type(ba)

bytearray

In [36]:
ba[1]

98

In [37]:
type(ba[1])

int

In [38]:
ba[1] = 255

In [39]:
ba[1] = 256

ValueError: byte must be in range(0, 256)

In [41]:
bytes(ba)

b'a\xffc'

### `bytes` en strings

Je kunt een string omzetten in een `bytes`-waarde, en omgekeerd. Je moet dan opgeven welke codering gebruikt wordt; meestal is dat `utf-8-`.